### Goal:
#### - Analyzing the impact of Neighborhood Characteristic to Covid19 spread
#### - Verifying social distancing necessity in reducing the number of infection
*Data as of 22nd April 2020

In [50]:
import pandas as pd
import numpy as np
import requests
import json
from pandas import json_normalize

from geopy.geocoders import Nominatim

import matplotlib

from sklearn.cluster import KMeans

import folium

### Analyzing Covid 19 data in Toronto
*source: https://www.toronto.ca/home/covid-19/covid-19-latest-city-of-toronto-news/covid-19-status-of-cases-in-toronto/

Retrieving the number of cases per day

In [51]:
#!conda install -c anaconda xlrd

covfile = r'CityofToronto_COVID-19_Data.xlsx'
cov_tor = pd.read_excel(covfile, sheet_name='Cases by Episode Date')
cov_tor.head(60)

,Episode Date,Case Count
0,2020-04-21,2
1,2020-04-20,28
2,2020-04-19,20
3,2020-04-18,36
4,2020-04-17,86
5,2020-04-16,149
6,2020-04-15,190
7,2020-04-14,214
8,2020-04-13,317
9,2020-04-12,128


In [52]:
start = cov_tor['Episode Date'].min()
last = cov_tor['Episode Date'].max()
num = last - start
print('This data is from {} to {}, {} days'.format(start,last,num))


This data is from 2020-01-21 00:00:00 to 2020-04-21 00:00:00, 91 days 00:00:00 days


#### Specifying lockdown time in Ontario
*source: https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Ontario

In [53]:
ON_lockdown_1 = pd.to_datetime('2020-03-17')
ON_lockdown_2 = pd.to_datetime('2020-03-28')

#### Analyzing Covid19 status in Ontario
Source: 
- https://data.ontario.ca/dataset/status-of-covid-19-cases-in-ontario
- https://www.ontario.ca/page/2019-novel-coronavirus#section-4
- https://files.ontario.ca/moh-covid-19-report-en-2020-04-22.pdf
- https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200422-sitrep-93-covid-19.pdf?sfvrsn=35cf80d7_4

In [54]:
cov_on_file = r'covidtesting.csv'
cov_on = pd.read_csv(cov_on_file)
cov_on.head()

,Reported Date,Confirmed Negative,Presumptive Negative,Presumptive Positive,Confirmed Positive,Resolved,Deaths,Total Cases,Total patients approved for testing as of Reporting Date,Total tests completed in the last day,Under Investigation,Number of patients hospitalized with COVID-19,Number of patients in ICU with COVID-19,Number of patients in ICU on a ventilator with COVID-19
0,2020-01-26,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN
1,2020-01-27,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN
2,2020-01-28,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,11,NaN,NaN,NaN
3,2020-01-30,NaN,NaN,0.0,2.0,NaN,NaN,2.0,NaN,NaN,27,NaN,NaN,NaN
4,2020-01-31,NaN,NaN,0.0,2.0,NaN,NaN,2.0,NaN,NaN,17,NaN,NaN,NaN


In [55]:
cov_on.columns

Index(['Reported Date', 'Confirmed Negative', 'Presumptive Negative',
       'Presumptive Positive', 'Confirmed Positive', 'Resolved', 'Deaths',
       'Total Cases',
       'Total patients approved for testing as of Reporting Date',
       'Total tests completed in the last day', 'Under Investigation',
       'Number of patients hospitalized with COVID-19',
       'Number of patients in ICU with COVID-19',
       'Number of patients in ICU on a ventilator with COVID-19'],
      dtype='object')

#### Analyzing confirmed positive cases in Ontario
*source: https://data.ontario.ca/dataset/confirmed-positive-cases-of-covid-19-in-ontario

In [56]:
cov_pos_file = r'conposcovidloc.csv'
cov_pos = pd.read_csv(cov_pos_file)
cov_pos.head()

,Row_ID,Accurate_Episode_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,1,2020-01-22,50s,FEMALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
1,2,2020-01-21,50s,MALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
2,3,2020-01-24,20s,FEMALE,Travel-Related,Resolved,Middlesex-London Health Unit,50 King Street,London,N6A 5L7,www.healthunit.com,42.981468,-81.254016
3,4,2020-02-05,20s,FEMALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
4,5,2020-02-16,60s,FEMALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358


In [57]:
cov_pos.shape

(12245, 13)

In [58]:
cov_pos_filtered = cov_pos[['Row_ID','Accurate_Episode_Date','Case_AcquisitionInfo','Outcome1','Reporting_PHU_City','Reporting_PHU_Latitude','Reporting_PHU_Longitude']].set_index(['Row_ID'])

In [59]:
cov_pos_filtered.head(10)

,Accurate_Episode_Date,Case_AcquisitionInfo,Outcome1,Reporting_PHU_City,Reporting_PHU_Latitude,Reporting_PHU_Longitude
Row_ID,,,,,,
1,2020-01-22,Travel-Related,Resolved,Toronto,43.656591,-79.379358
2,2020-01-21,Travel-Related,Resolved,Toronto,43.656591,-79.379358
3,2020-01-24,Travel-Related,Resolved,London,42.981468,-81.254016
4,2020-02-05,Travel-Related,Resolved,Toronto,43.656591,-79.379358
5,2020-02-16,Travel-Related,Resolved,Toronto,43.656591,-79.379358
6,2020-02-20,Contact of a confirmed case,Resolved,Toronto,43.656591,-79.379358
7,2020-02-24,Travel-Related,Resolved,Toronto,43.656591,-79.379358
8,2020-02-25,Travel-Related,Resolved,Newmarket,44.048023,-79.480239
9,2020-02-20,Travel-Related,Resolved,Toronto,43.656591,-79.379358


In [60]:
cov_pos['Reporting_PHU_City'].unique()

array(['Toronto', 'London', 'Newmarket', 'Whitby', 'Waterloo',
       'Mississauga', 'Oakville', 'Ottawa', 'Sudbury', 'Hamilton',
       'Cornwall', 'Barrie', 'Thorold', 'Port Hope', 'Stratford',
       'Kenora', 'Peterborough', 'Owen Sound', 'Guelph', 'Brantford',
       'Windsor', 'Kingston', 'Sault Ste. Marie', 'Timmins', 'Chatham',
       'Belleville', 'St. Thomas', 'New Liskeard', 'North Bay',
       'Point Edward', 'Thunder Bay', 'Brockville', 'Simcoe', 'Pembroke'],
      dtype=object)

#### Get the list of Municipality in Ontario

In [61]:
from bs4 import BeautifulSoup
import urllib.request
mun_url = 'https://en.wikipedia.org/wiki/List_of_municipalities_in_Ontario'
mun_page = urllib.request.urlopen(mun_url)
soup = BeautifulSoup(mun_page,'html.parser')
#print(soup.prettify())

In [62]:
mun_table = soup.find_all("table",class_="wikitable sortable")
A=[]
B=[]
C=[]
D=[]

for row in mun_table[1].find_all("tr"): #there are 2 table in the page, get the second table
    cells = row.find_all("td")
    if len(cells) == 9:
        A.append(cells[0].find(text=True).rstrip()) #get the lower-tier municipality name
        B.append(cells[4].find(text=True).rstrip()) #get the population number
        C.append(cells[7].find(text=True).rstrip()) #get the area size
        D.append(cells[8].find(text=True).rstrip()) #get the density

In [63]:
mun_on = pd.DataFrame(A,columns=['Municipality'])

In [64]:
mun_on.head(30)
mun_on.shape

(414, 1)

In [65]:
mun_on['Population']=B
mun_on['Area']=C
mun_on['Density']=D

In [66]:
mun_on.head()

,Municipality,Population,Area,Density
0,Addington Highlands,"2,323","1,328.32",1.7/km
1,Adelaide-Metcalfe,"2,990",331.46,9.0/km
2,Adjala-Tosorontio,"10,975",372.34,29.5/km
3,Admaston/Bromley,"2,935",524.06,5.6/km
4,Ajax,"119,677",67.00,"1,786.2/km"


In [67]:
mun_on.replace(to_replace=',',value='',regex=True,inplace=True)
mun_on.replace(to_replace='/km',value='',regex=True,inplace=True)

In [68]:
mun_on['Population'] = mun_on['Population'].astype(int)
mun_on['Area'] = mun_on['Area'].astype(float)
mun_on['Density'] = mun_on['Density'].astype(float)

In [69]:
mun_on.head(10)

,Municipality,Population,Area,Density
0,Addington Highlands,2323,1328.32,1.7
1,Adelaide-Metcalfe,2990,331.46,9.0
2,Adjala-Tosorontio,10975,372.34,29.5
3,Admaston/Bromley,2935,524.06,5.6
4,Ajax,119677,67.00,1786.2
5,Alberton,969,116.66,8.3
6,Alfred and Plantagenet,9680,392.31,24.7
7,Algonquin Highlands,2351,1007.20,2.3
8,Alnwick/Haldimand,6869,398.45,17.2
9,Amaranth,4079,264.58,15.4


In [70]:
mun_on.shape

(414, 4)

In [71]:
def getlatlang(city):
    geolocator = Nominatim(user_agent="tor_explorer")
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    print('Coordinate of {} are {},{}'.format(city,latitude, longitude))
    return latitude,longitude

In [72]:
mun_sorted = mun_on.sort_values(by=['Population'],ascending=False).reset_index(drop=True)
mun_sorted.head()

,Municipality,Population,Area,Density
0,Toronto,2731571,630.20,4334.5
1,Ottawa,934243,2790.30,334.8
2,Mississauga,721599,292.43,2467.6
3,Brampton,593638,266.36,2228.7
4,Hamilton,536917,1117.29,480.6


In [73]:
cov_pos_loc = cov_pos['Reporting_PHU_City'].unique()

In [74]:
latlist = []
longlist = []

for m in cov_pos_loc:
    result = getlatlang(m + ', ON')
    latlist.append(result[0])
    longlist.append(result[1])

Coordinate of Toronto, ON are 43.6534817,-79.3839347
Coordinate of London, ON are 42.9836747,-81.2496068
Coordinate of Newmarket, ON are 44.056258,-79.461708
Coordinate of Whitby, ON are 43.899871,-78.940201
Coordinate of Waterloo, ON are 43.466874,-80.524635
Coordinate of Mississauga, ON are 43.590338,-79.645729
Coordinate of Oakville, ON are 43.447436,-79.666672
Coordinate of Ottawa, ON are 45.421106,-75.690308
Coordinate of Sudbury, ON are 46.49272,-80.991211
Coordinate of Hamilton, ON are 43.2560802,-79.8728583
Coordinate of Cornwall, ON are 45.0184417,-74.728703
Coordinate of Barrie, ON are 44.3893113,-79.6901736
Coordinate of Thorold, ON are 43.1179352,-79.202235
Coordinate of Port Hope, ON are 43.9515755,-78.2939704
Coordinate of Stratford, ON are 52.1927885,-1.7063668
Coordinate of Kenora, ON are 49.766215,-94.4893528
Coordinate of Peterborough, ON are 44.3048009,-78.3199496
Coordinate of Owen Sound, ON are 44.5668092,-80.9441252
Coordinate of Guelph, ON are 43.5460516,-80.2493

In [75]:
len(latlist)

34

In [76]:
cov_city = pd.DataFrame(cov_pos_loc,columns=['Reporting PHU City'])

In [77]:
cov_city['Latitude']=latlist
cov_city['Longitude']=longlist
cov_city

,Reporting PHU City,Latitude,Longitude
0,Toronto,43.653482,-79.383935
1,London,42.983675,-81.249607
2,Newmarket,44.056258,-79.461708
3,Whitby,43.899871,-78.940201
4,Waterloo,43.466874,-80.524635
5,Mississauga,43.590338,-79.645729
6,Oakville,43.447436,-79.666672
7,Ottawa,45.421106,-75.690308
8,Sudbury,46.492720,-80.991211
9,Hamilton,43.256080,-79.872858


In [82]:
mun_sorted.to_csv('municipal.csv')

In [83]:
cov_city.to_csv('cov_city.csv')

In [80]:
total_can = 40190
total_on = 12245

In [81]:
total_on / total_can

0.3046777805424235